In [1]:
#HERE YOU FIND ALL THE LIBRARYS FOR THIS PROJECT
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler

#this librarys are for the gpu usage
import tensorflow as tf
from keras.utils.training_utils import multi_gpu_model
from tensorflow.python.client import device_lib

Using TensorFlow backend.


In [2]:
"""
Setup multi GPU usage

Example usage:
model = Sequential()
...
multi_model = multi_gpu_model(model, gpus=num_gpu)
multi_model.fit()

About memory usage:
https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
"""

# IMPORTANT: Tells tf to not occupy a specific amount of memory
from keras.backend.tensorflow_backend import set_session  
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU  
sess = tf.Session(config=config)  
set_session(sess)  # set this TensorFlow session as the default session for Keras.


# getting the number of GPUs 
def get_available_gpus():
   local_device_protos = device_lib.list_local_devices()
   return [x.name for x in local_device_protos if x.device_type    == 'GPU']
num_gpu = len(get_available_gpus())
print('Amount of GPUs available: %s' % num_gpu)

Amount of GPUs available: 4


In [3]:
#SELECT THE DATA TO USE
df = pd.read_csv('//datc//opschaler//combined_gas_smart_weather_dfs//processed//all_dwellings_combined_hour.csv', delimiter='\t', parse_dates=['datetime'])
df = df.set_index(['datetime'])
df = df.dropna()

df['gasPower'] = df['gasPower']*10

In [4]:
#RESAMPLE TO ONE DAY DATA
df_day = df.resample('1D').mean()
df_day = df_day.dropna()

In [5]:
df_day['hour'] = df_day.index.hour #create column containing the hour
df_day['dayofweek'] = df_day.index.dayofweek #Create column containing the day of the week

columns_to_cat = ['hour', 'dayofweek']
df_day[columns_to_cat] = df_day[columns_to_cat].astype('category') # change datetypes to category

df_day = pd.get_dummies(df_day, columns=columns_to_cat) # One hot encoding the categories

df_day['T-1'] = df_day['T'].shift(1)
df_day = df_day.dropna()

In [6]:
df_day.head()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn,gasMeter,DD,DR,FF,...,gasPower,hour_0,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,T-1
datetime,,,,,,,,,,,,,,,,,,,,,
2017-02-23,1558.435883,0.0,1805.226395,0.0,173.374907,0.0,1685.894847,239.148194,265.916667,11.608438,...,4.857127,1,0,0,0,1,0,0,0,9.236111
2017-02-24,1561.837229,0.0,1806.231979,0.0,246.992340,0.0,1694.890589,278.093543,45.766667,5.032826,...,2.768300,1,0,0,0,0,1,0,0,8.590000
2017-02-25,1564.963013,0.0,1808.506632,0.0,150.144606,0.0,1698.313894,202.265418,14.750000,6.502271,...,0.171661,1,0,0,0,0,0,1,0,4.746250
2017-02-26,1564.963013,0.0,1817.410950,0.0,679.171475,0.0,1706.148254,208.350207,82.633334,7.540049,...,5.030009,1,0,0,0,0,0,0,1,5.606806
2017-02-27,1566.147619,0.0,1828.823593,0.0,253.032893,0.0,1711.548060,192.541318,111.916667,7.891396,...,1.315816,1,1,0,0,0,0,0,0,8.688750


In [8]:
X = df_day[['T']]
y = df_day[['gasPower']]

look_back = 5#*24 # D -> 5, H -> 5*24
num_features = 1
output_dim = 1
train_size = 0.7

X_train, y_train, X_test, y_test = df_to_cnn_rnn_format(df=df_day, train_size=train_size, look_back=look_back, target_column='gasPower', scale_X=True)

NameError: name 'df_to_cnn_rnn_format' is not defined